## Шаг 1: выбор темы

### Тема проекта:
Анализ рынка аренды квартир в Москве на основе данных с сайта Циан.

### Актуальность проекта:
Многие студенты как раз к третьему, четвертому курсу начинают задумываться о съеме квартиры. В москве этот рынок огромен, сложно просматривать все объявления, выцепляя наилучшие варианты. В этом проекте мы хотим проанализировать рынок аренды квартир в москве на основе данных Циан. Также мы хотим обучить модель, которая будет определять квартиры, которые имеют цену ниже рыночной. Тогда можно было бы определить механизм выделения наиболее интересных вариантов квартир. Также хотим рассмотреть различные инетересные зависимости, получить ответы на вопросы, которые возникнут у нас в процессе.


## Шаг 2: сбор данных (максимум 20)

In [23]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import random
import time

def get_link_announcement(page, count_links):
    
    response = requests.get(f'https://www.cian.ru/cat.php?deal_type=rent&engine_version=2&offer_type=flat&p={page}&region=1&room1=1&room2=1&room9=1&type=4')
    count_links_new = 0

    if response.status_code == 200:
        tree_link = BeautifulSoup(response.text, 'html.parser')
        
        # Находим все элементы, где находится ссылка на объявление
        announcement_links = tree_link.find_all('a', class_="_93444fe79c--media--9P6wN")
        
        # Ссобираем ссылки в отдельный список
        link_list = [link.get('href') for link in announcement_links]
        count_links_new = len(link_list)
        count_links +=  count_links_new
        
        # Записываем все ссылки в файл
        with open('D:\\Артем\\уник\\Питон\\ИТОГ проект\\ИТОГ 2\\Data_links — итог 2.txt', 'a', encoding='utf-8') as file:
            for link in  link_list:
                file.write(link + '\n')
                
    elif response.status_code == 429:
        try:
            retry_after = int(response.headers["Retry-After"])
            print(f'Слишком много запросов. Повторная попытка через {retry_after} секунд.')
            time.sleep(retry_after)
        except Exception as e:
            print(f'Ошибка получения времени ожидания:', e)
            time.sleep(10)  # Ожидание по умолчанию
        page -= 1
    
    return count_links,  count_links_new, page 



In [12]:
#Спарим данные ссылки на все объявления, которые можем спарсить без использования Selenium (по методу проб и ошибок выяснилось, что у Циан всего 54 страицы можно перебрать по ссылки, а дальше на странице 54 нужно добавлять данные через кнопку 'Показать еще')

count_links = 0
page = 0
percing = True

total_pages = 54
pbar = tqdm(total=total_pages, desc='Pages received')
while page <= total_pages and percing:
    page += 1
    count_links, count_links_new, page = get_link_announcement(page, count_links)
    time.sleep(2 + random.uniform(0, 2))
    if count_links % 1000 == 0 and count_links > 0:
        time.sleep(180)
    pbar.update(1)
del pbar
print(f'Всего было добавлено {count_links} ссылок и {page} страниц')

Pages received:  89%|████████▉ | 48/54 [03:49<00:25,  4.25s/it]

Ошибка получения времени ожидания: 'retry-after'


Pages received:  94%|█████████▍| 51/54 [04:08<00:14,  4.99s/it]

Ошибка получения времени ожидания: 'retry-after'


Pages received: 100%|██████████| 54/54 [04:26<00:00,  5.17s/it]

...

Ошибка получения времени ожидания: 'retry-after'


Pages received: 59it [05:18, 10.30s/it]

Ошибка получения времени ожидания: 'retry-after'


Pages received: 61it [05:40,  5.59s/it]

Всего было добавлено 1537 ссылок и 55 страниц


Но после страницы 54 необходимо прокручивать струнику через 'Показать еще', что оставляет нам только один вариант - силениум

In [25]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time

url = 'https://www.cian.ru/cat.php?deal_type=rent&engine_version=2&offer_type=flat&p=54&region=1&room1=1&room2=1&room9=1&type=4'

# открываем браузер и переходим по ссылке
driver = webdriver.Chrome()
driver.get(url)

# Запишем, сколько раз мы нажмем "Показать еще". Если нажмем n раз, то получим n*28 объявлений 
max_show = 70
all_links = set()

for _ in range(max_show):
    # находим текущие объявления
    links = driver.find_elements(By.CLASS_NAME, '_93444fe79c--media--9P6wN')
    
    # извлекаем ссылки на объявления
    new_links = [flat.get_attribute('href') for flat in links]
    
    previous_count = len(all_links)
    all_links.update(new_links)
    
    if len(all_links) == previous_count:
        break
    
    try:
        # ищем "Показать еще" и листаем к ней
        more_links = driver.find_element(By.CSS_SELECTOR, 'a._93444fe79c--more-button--nqptt')
        driver.execute_script("arguments[0].scrollIntoView(true);", more_links)
        
        # ждем немного для полной загрузки
        time.sleep(1)
        
        # создаем объект ActionChains
        actions = ActionChains(driver)
        actions.move_to_element(more_links).click().perform()
        
        # ждем, чтобы страница обновилась
        time.sleep(random.uniform(3, 5))
    except Exception as e:
        print(f"Кнопка 'Показать ещё' не найдена или не кликается: {e}")
        break

link_list_selenium = []
# выводим все собранные ссылки
for link in tqdm(all_links):
    link_list_selenium.append(link)
with open('D:\\Артем\\уник\\Питон\\ИТОГ проект\\ИТОГ 2\\Data_links — итог 2.txt', 'a', encoding='utf-8') as file:
    for link in  link_list_selenium:
        file.write(link + '\n')

# закрываем браузер
driver.quit()

Кнопка 'Показать ещё' не найдена или не кликается: Message: no such element: Unable to locate element: {"method":"css selector","selector":"a._93444fe79c--more-button--nqptt"}
  (Session info: chrome=125.0.6422.78); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6A12D1F22+60322]
	(No symbol) [0x00007FF6A124CE99]
	(No symbol) [0x00007FF6A1107EBA]
	(No symbol) [0x00007FF6A1157676]
	(No symbol) [0x00007FF6A115773C]
	(No symbol) [0x00007FF6A119E967]
	(No symbol) [0x00007FF6A117C25F]
	(No symbol) [0x00007FF6A119BC80]
	(No symbol) [0x00007FF6A117BFC3]
	(No symbol) [0x00007FF6A1149617]
	(No symbol) [0x00007FF6A114A211]
	GetHandleVerifier [0x00007FF6A15E946D+3301613]
	GetHandleVerifier [0x00007FF6A1633693+3605267]
	GetHandleVerifier [0x00007FF6A1629410+3563664]
	GetHandleVerifier [0x00007FF6A13842F6+790390]
	(No symbol) [0x00007FF6A12574DF]
	(No symbol) [0x

100%|██████████| 134/134 [00:00<?, ?it/s]


In [9]:
#сформируем список из файла, куда спарсили ссылки, и проверим, сколько спарсилось уникальных ссылок
links_1 = []
with open('D:\\Артем\\уник\\Питон\\ИТОГ проект\\ИТОГ 2\\Data_links — итог 2.txt', 'r', encoding='utf-8') as file:
    for line in file:
        line = line.strip()
        links_1.append(line)
print(len(set(links_1)))


1626


Теперь спарсим данные по каждой ссылке

In [7]:
import requests
from bs4 import BeautifulSoup
import random
import time
import numpy as np
import pandas as pd
from tqdm import tqdm

info = {
    'Link': [], 'Цена': [], 'Пешком до метро': [], 'Станция метро': [], 'Адрес': [], 
    'Общая площадь': [], 'Жилая площадь': [], 'Площадь кухни': [], 'Высота потолков': [], 'Санузел': [], 
    'Балкон/лоджия': [], 'Ремонт': [], 'Год постройки': [], 'Строительная серия': [], 'Тип дома': [], 
    'Аварийность': [], 'доп инфа': []
}


info_additions = {'Link': [], 'В квартире есть': [], 'Залог': [], 'Предоплата': [], 'Срок аренды': []}

def appartment_data(link):
    try:
        response = requests.get(link)
        if response.status_code == 200:
            info['Link'].append(link)
            info_additions['Link'].append(link)

            tree_appartment = BeautifulSoup(response.text, 'html.parser')
            main_parametrs = tree_appartment.find_all(
                'span', class_='a10a3f92e9--color_black_100--Ephi7 a10a3f92e9--lineHeight_6u--cedXD a10a3f92e9--fontWeight_bold--BbhnX a10a3f92e9--fontSize_16px--QNYmt a10a3f92e9--display_block--KYb25 a10a3f92e9--text--e4SBY')
            parametrs_text = [param.text.replace('\xa0', ' ').replace(' м²', '') for param in main_parametrs]

            price_amount = tree_appartment.find(
                'span', class_='a10a3f92e9--color_black_100--Ephi7 a10a3f92e9--lineHeight_9u--limEs a10a3f92e9--fontWeight_bold--BbhnX a10a3f92e9--fontSize_28px--P1gR4 a10a3f92e9--display_block--KYb25 a10a3f92e9--text--e4SBY')
            if price_amount is not None:
                price_amount = price_amount.text.replace('\xa0', '').replace('/', ' ').replace('₽', '').split()[0]
            else:
                price_amount = np.nan
            info['Цена'].append(price_amount)

            room_amount = tree_appartment.find('h1', class_='a10a3f92e9--title--vlZwT')
            if room_amount is not None:
                room_amount = room_amount.text
            else:
                room_amount = np.nan
            info['доп инфа'].append(room_amount)

            walking_to_metro = tree_appartment.find('span', class_='a10a3f92e9--underground_time--YvrcI')
            if walking_to_metro is not None:
                walking_to_metro = walking_to_metro.text
            else:
                walking_to_metro = np.nan
            info['Пешком до метро'].append(walking_to_metro)

            metro = tree_appartment.find('a', class_='a10a3f92e9--underground_link--VnUVj')
            if metro is not None:
                metro = metro.text
            else:
                metro = np.nan
            info['Станция метро'].append(metro)

            adress = tree_appartment.find_all('a', class_='a10a3f92e9--address--SMU25')
            if adress:
                adress = [part.text for part in adress]
            else:
                adress = np.nan
            info['Адрес'].append(adress)

            about_appartments_name = tree_appartment.find_all(
                'p', class_='a10a3f92e9--color_gray60_100--mYFjS a10a3f92e9--lineHeight_6u--cedXD a10a3f92e9--fontWeight_normal--JEG_c a10a3f92e9--fontSize_16px--QNYmt a10a3f92e9--display_block--KYb25 a10a3f92e9--text--e4SBY a10a3f92e9--text_letterSpacing__0--cQxU5')
            about_appartments_name = [about.text for about in about_appartments_name]

            about_appartments_information = tree_appartment.find_all(
                'p', class_='a10a3f92e9--color_black_100--Ephi7 a10a3f92e9--lineHeight_6u--cedXD a10a3f92e9--fontWeight_normal--JEG_c a10a3f92e9--fontSize_16px--QNYmt a10a3f92e9--display_block--KYb25 a10a3f92e9--text--e4SBY a10a3f92e9--text_letterSpacing__0--cQxU5')
            about_appartments_information = [about.text for about in about_appartments_information]

            for elem in list(info.keys())[5:-1]:
                if elem in about_appartments_name:
                    info[elem].append(about_appartments_information[about_appartments_name.index(elem)])
                else:
                    info[elem].append(np.nan)

            apartment_additions = tree_appartment.find_all('div', class_='a10a3f92e9--wrapper--Ycswz')
            if apartment_additions:
                apartment_additions = [addit.text for addit in apartment_additions]
            else:
                apartment_additions = np.nan
            info_additions['В квартире есть'].append(apartment_additions)

            percent = tree_appartment.find_all('div', class_='a10a3f92e9--item--iWTsg')
            for elem in percent:
                label = elem.find('span', class_='a10a3f92e9--color_black_100--Ephi7')
                if label:
                    value = elem.find('span', class_='a10a3f92e9--color_black_100--Ephi7', string=lambda t: 'Залог' not in t and 'Комиссия' not in t and 'Предоплата' not in t and 'Срок аренды' not in t)
                    if value:
                        value = value.text
                    else:
                        value = np.nan
                    if 'Залог' in label.text:
                        info_additions['Залог'].append(value)
                    elif 'Предоплата' in label.text:
                        info_additions['Предоплата'].append(value)
                    elif 'Срок аренды' in label.text:
                        info_additions['Срок аренды'].append(value)

        elif response.status_code == 429:
            retry_after = response.headers.get("Retry-After")
            wait_time = int(retry_after) if retry_after is not None else random.uniform(3, 7)
            time.sleep(wait_time)
            return appartment_data(link)

    except Exception as e:
        time.sleep(20)
        return appartment_data(link)

    return ()

In [10]:
#выше уже считывали данные из файла, куда добавили все ссылки на объявления, так что дублировать не буду
for i in tqdm(links_1):
    appartment_data(i)
    time.sleep(random.uniform(1, 3))


 31%|███       | 599/1940 [2:07:26<4:45:18, 12.77s/it]


KeyboardInterrupt: 

In [11]:
for i in tqdm(links_1[599:]):
    appartment_data(i)
    time.sleep(random.uniform(1, 3))

 49%|████▉     | 663/1341 [5:54:29<6:02:30, 32.08s/it]   


KeyboardInterrupt: 

In [12]:
for i in tqdm(links_1[1262:]):
    appartment_data(i)
    time.sleep(random.uniform(1, 3))


100%|█████████▉| 677/678 [2:25:06<00:12, 12.86s/it]


KeyboardInterrupt: 

Из за нестабильного интернета пришлось несколько раз запускать парсинг, но это никак не сказывается на эффективности кода, потере данных, а также на возможности спарсить все без приостановок при стабильном интернете

In [23]:
info_copy = info.copy()
info_additions_copy = info_additions.copy()

In [24]:
for i in info_additions.keys():
    print(len(info_additions[i]))

1933
1933
1933
1908
1933


Так как парсинг достаточно продолжительный процесс, то было решено удалить столбец 'Предоплата', что обеспечит единую размерность и возможность работать с таблицами библиотеки pandas соответственно

In [ ]:
info_additions.pop('Предоплата', None)


In [28]:
df_additions = pd.DataFrame(info_additions)
df_additions.head()

,Link,В квартире есть,Залог,Срок аренды
0,https://www.cian.ru/rent/flat/278216280/,[ХолодильникПосудомоечная машинаСтиральная маш...,1 000 000 ₽,от года
1,https://www.cian.ru/rent/flat/301171666/,[ХолодильникПосудомоечная машинаСтиральная маш...,300 000 ₽,от года
2,https://www.cian.ru/rent/flat/257204026/,[ХолодильникСтиральная машинаТелевизорИнтернет...,32 000 ₽,от года
3,https://www.cian.ru/rent/flat/289432753/,[ХолодильникПосудомоечная машинаСтиральная маш...,350 000 ₽,от года
4,https://www.cian.ru/rent/flat/278216280/,[ХолодильникПосудомоечная машинаСтиральная маш...,1 000 000 ₽,от года


In [29]:
df_info = pd.DataFrame(info)
df_info.head()

,Link,Цена,Пешком до метро,Станция метро,Адрес,Общая площадь,Жилая площадь,Площадь кухни,Высота потолков,Санузел,Балкон/лоджия,Ремонт,Год постройки,Строительная серия,Тип дома,Аварийность,доп инфа
0,https://www.cian.ru/rent/flat/278216280/,ID,1 мин.,Охотный ряд,"[Москва, ЦАО, р-н Тверской, ул. Охотный Ряд, 2]",90,NaN,NaN,NaN,"1 совмещенный, 1 раздельный",NaN,Дизайнерский,2015,NaN,Монолитный,Нет,"Сдается 2-комн. апартаменты, 90 м²"
1,https://www.cian.ru/rent/flat/301171666/,300000,4 мин.,Деловой центр,"[Москва, ЦАО, р-н Пресненский, 1-й Красногвард...",85,40,17,"3,2 м",2 совмещенных,NaN,Дизайнерский,NaN,NaN,NaN,NaN,"Сдается 2-комн. апартаменты, 85 м²"
2,https://www.cian.ru/rent/flat/257204026/,32000,10 мин.,Черкизовская,"[Москва, Большая Черкизовская ул., 22К1]",40,"18,3","7,6","2,48 м",1 совмещенный,NaN,Косметический,1974,ii-68,Панельный,Нет,"Сдается 1-комн. квартира, 40 м²"
3,https://www.cian.ru/rent/flat/289432753/,350000,4 мин.,Деловой центр,"[Москва, ЦАО, р-н Пресненский, 1-й Красногвард...",90,45,40,"4,5 м",2 совмещенных,NaN,Евроремонт,2013,NaN,Монолитный,Нет,"Сдается 2-комн. апартаменты, 90 м²"
4,https://www.cian.ru/rent/flat/278216280/,ID,1 мин.,Охотный ряд,"[Москва, ЦАО, р-н Тверской, ул. Охотный Ряд, 2]",90,NaN,NaN,NaN,"1 совмещенный, 1 раздельный",NaN,Дизайнерский,2015,NaN,Монолитный,Нет,"Сдается 2-комн. апартаменты, 90 м²"


In [32]:
#Сохраним данные
df_info.to_csv('D:\\Артем\\уник\\Питон\\ИТОГ проект\\ИТОГ 2\\Parcer data info.txt', index=False, encoding='utf-8')
df_additions.to_csv('D:\\Артем\\уник\\Питон\\ИТОГ проект\\ИТОГ 2\\Parcer data additions.txt', index=False, encoding='utf-8')


___________________________________________________________________________________________________________________________________________________________________________________________________________________________________________